# **Deep Learning Project** - Aerial Perspective Object Detection
SEP 769 - Group 2

In [1]:
#About semantic segmentation: https://www.jeremyjordan.me/semantic-segmentation/

from google.colab import drive
drive.mount('/content/gdrive')

%cd '/content/gdrive/Shareddrives/SEP_769'

%ls

#!unzip '/content/gdrive/Shareddrives/SEP_769/archive.zip'


#Variable Locations - Kaggle
#Original Images: /content/gdrive/dataset/emantic_drone_dataset/original_images
#Original Images as Greyscale Segmentation Maps: /content/gdrive/dataset/semantic_drone_dataset/label_images_semantic
#Original Images as RGB Segmentation Maps: /content/gdrive/dataset/RGB_color_image_masks/RGB_color_image_masks

#Variable Locations - Requested Dataset
#Original Images: /content/gdrive/dataset/semantic_drone_dataset/training_set/images
#Original Images as RGB Segmentation Maps: /content/gdrive/dataset/semantic_drone_dataset/training_set/gt/semantic/label_images 
#Original Image labels and locations: /content/gdrive/dataset/semantic_drone_dataset/training_set/gt/semantic/label_me_xml


Mounted at /content/gdrive
/content/gdrive/Shareddrives/SEP_769
archive.zip         dataset/                semantic_drone_dataset/
class_dict_seg.csv  RGB_color_image_masks/
